In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Importação de Bibliotecas.

In [ ]:
import os
import time
import cv2
import numpy as np
from skimage import segmentation
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

#Parametros da Rede

In [ ]:
class Args(object):
    input_image_path = '/content/drive/MyDrive/EXPA/ML-Segmentation/Etapa1/Sandstone-modelo-etapa1.png'  # image/coral.jpg image/tiger.jpg
    train_epoch = 40
    mod_dim1 = 16  # Número de filtros
    mod_dim2 = 8  # Número de filtros
    gpu_id = 0 # Número da GPU

    min_label_num = 4  # if the label number small than it, break loop
    max_label_num = 50  # if the label number small than it, start to show result image.

# Arquitetura da Rede
> nn.Conv2d : https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html


In [ ]:
class MyNet(nn.Module):
    def __init__(self, inp_dim, mod_dim1, mod_dim2):
        super(MyNet, self).__init__()

        self.seq = nn.Sequential(
            nn.Conv2d(inp_dim, mod_dim1, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(mod_dim1),
            nn.ReLU(inplace=True),

            nn.Conv2d(mod_dim1, mod_dim2, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(mod_dim2),
            nn.ReLU(inplace=True),

            nn.Conv2d(mod_dim2, mod_dim1, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(mod_dim1),
            nn.ReLU(inplace=True),

            nn.Conv2d(mod_dim1, mod_dim2, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(mod_dim2),
        )

    def forward(self, x):
        return self.seq(x)

# Carrega os Parametros da rede e a imagem de input.

In [ ]:
start_time0 = time.time()

args = Args()
torch.cuda.manual_seed_all(1943)
np.random.seed(1943)
os.environ['CUDA_VISIBLE_DEVICES'] = str(args.gpu_id)  # choose GPU:0
image = cv2.cvtColor(cv2.imread(args.input_image_path), cv2.COLOR_BGR2RGB) # Reand and convert to RGB
h,l,d=image.shape
perc=0.1
image = image[int(h*perc):int(h-h*perc),int(l*perc):int(l-l*perc)]



# Filtragem da imagem de Input.

> CV2 Filters: https://docs.opencv.org/4.x/d4/d86/group__imgproc__filter.html 

In [ ]:
''' image filter'''
kSize=3
image = cv2.medianBlur(src=image, ksize=kSize)
# image = cv2.GaussianBlur(image, (kSize,kSize),0)

# Definição dos Labels (segmentação) da imagem de Input.
> Felzenszwalb: https://scikit-image.org/docs/stable/api/skimage.segmentation.html#skimagesegmentation.felzenszwalb

> SLIC: https://scikit-image.org/docs/stable/api/skimage.segmentation.html#skimage.segmentation.slic

In [ ]:
'''segmentation ML'''
seg_map = segmentation.felzenszwalb(image, scale=32, sigma=0.5, min_size=64)
# seg_map = segmentation.slic(image, n_segments=10000, compactness=100)
seg_map = seg_map.flatten()
seg_lab = [np.where(seg_map == u_label)[0]
            for u_label in np.unique(seg_map)]

# Inicialização do Modelo da rede.

In [ ]:
'''train init'''
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

tensor = image.transpose((2, 0, 1))
tensor = tensor.astype(np.float32) / 255.0
tensor = tensor[np.newaxis, :, :, :]
tensor = torch.from_numpy(tensor).to(device)

model = MyNet(inp_dim=3, mod_dim1=args.mod_dim1, mod_dim2=args.mod_dim2).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=5e-2, momentum=0.9)
# optimizer = torch.optim.RMSprop(model.parameters(), lr=1e-1, momentum=0.0)

image_flatten = image.reshape((-1, 3))
color_avg = np.random.randint(255, size=(args.max_label_num, 3))
show = image

# Loop de Treinamento do modelo



In [ ]:
'''train loop'''
start_time1 = time.time()
model.train()
for batch_idx in range(args.train_epoch):
    '''forward'''
    optimizer.zero_grad()
    output = model(tensor)[0]
    output = output.permute(1, 2, 0).view(-1, args.mod_dim2)
    target = torch.argmax(output, 1)
    im_target = target.data.cpu().numpy()

    '''refine'''
    for inds in seg_lab:
        u_labels, hist = np.unique(im_target[inds], return_counts=True)
        im_target[inds] = u_labels[np.argmax(hist)]

    '''backward'''
    target = torch.from_numpy(im_target)
    target = target.to(device)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()

    '''show image'''
    un_label, lab_inverse = np.unique(im_target, return_inverse=True, )
    if un_label.shape[0] < args.max_label_num:  # update show
        img_flatten = image_flatten.copy()
        if len(color_avg) != un_label.shape[0]:
            color_avg = [np.mean(img_flatten[im_target == label], axis=0, dtype=np.int) for label in un_label]
        for lab_id, color in enumerate(color_avg):
            img_flatten[lab_inverse == lab_id] = color
        show = img_flatten.reshape(image.shape)
    plt.imshow(show)   
    # plt.imshow(img_flatten)
    plt.show()
    # print(img_flatten)
    print('Loss:', batch_idx, loss.item())
    if len(un_label) < args.min_label_num: # Primeira Condição de Parada: Atingir O numero de labels igual ao definido em Args.
        break
    elif loss.item() < 0.2: # Segunda Condição de Parada: Atingir um valor de loss definido.
        break

Output hidden; open in https://colab.research.google.com to view.

#Salvamento da ultima imagem gerada pelo modelo.

In [ ]:
'''save'''
time0 = time.time() - start_time0
time1 = time.time() - start_time1
print('PyTorchInit: %.2f\nTimeUsed: %.2f' % (time0, time1))
cv2.imwrite(args.input_image_path[:-4].split('/')[-1] + "_seg_"+ str(batch_idx)+".jpg" , cv2.cvtColor(show, cv2.COLOR_BGR2RGB))
# cv2.imwrite("INSIRA O CAMINHO E NOME DO ARQUIVO QUE DESEJA SALVAR" , cv2.cvtColor(show, cv2.COLOR_BGR2RGB))

PyTorchInit: 27.31
TimeUsed: 25.94


True